In [1]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
from eumap.misc import find_files, nan_percentile, GoogleSheet, ttprint
from eumap.raster import read_rasters, save_rasters
from eumap.mapper import SpaceOverlay
import geopandas as gpd
from pathlib import Path
from minio import Minio
import rasterio
import pyproj
from shapely.geometry import Point
import requests
import warnings
warnings.filterwarnings('default')

# os.environ['PROJ_LIB'] = '/opt/conda/share/proj'
folder = '/mnt/primus/xuemeng_tmp_harbour/soc'

# /home/opengeohub/.local/bin

### check if what is need to be overlayed

In [2]:
from shapely.geometry import Point
from geopandas import gpd

# def get_data_to_be_overlayed(whole=False):
#     df4326 = gpd.read_file(f'{folder}/data/soil_overlay.4326.gpkg')
#     df3035 = gpd.read_file(f'{folder}/data/soil_overlay.3035.gpkg')
    
#     keys = ['sample_id', 'lat', 'lon', 'time', 'hzn_top', 'hzn_btm', 'ref']
#     if whole:
#         return df4326,df3035
#     else:
#         new4326 = gpd.read_file(f'{folder_path}/data/soil_overlay.4326_v2.gpkg')
#         merge4326 = pd.merge(df4326, new4326, on=keys, how='outer', indicator=True)
#         different_4326 = merge4326[merge4326['_merge'] != 'both']
        
#         new3035 = gpd.read_file(f'{folder_path}/data/soil_overlay.3035_v2.gpkg')
#         merge3035 = pd.merge(df3035, new3035, on=keys, how='outer', indicator=True)
#         different_3035 = merge3035[merge3035['_merge'] != 'both']
#         return different_4326,different_3035

# df4326, df3035 = get_data_to_be_overlayed(whole=True)     

# # create gpkg
added_covar = 0 # wether overlay from scratch or from overlayed version

if added_covar:
    df = pd.read_csv(f'{folder}/data/001_covar_overlayed.csv', low_memory=False)
else:
    df = pd.read_csv(f'{folder}/data/000_soil.full_qa.controlled.csv', low_memory=False)

geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

df4326 = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
df3035 = df4326.to_crs("EPSG:3035")


### generate overlay links
- read in the files specified by Google Sheet
- convert the files into readable linkes for overlay

In [3]:
# read in potential usable overlay files
key_file = '/mnt/inca/soc_eu_model/gaia-319808-913d36b5fca4.json'
url = 'https://docs.google.com/spreadsheets/d/1eIoPAvWM5jrhLrr25jwguAIR0YxOh3f5-CdXwpcOIz8/edit#gid=0'

gsheet = GoogleSheet(key_file, url)
covar = gsheet.covar


/home/opengeohub/.local/lib/python3.8/site-packages/gspread/auth.py:335: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  warnings.warn(


In [4]:
# function to generate file paths by year, and check if the urls are valid
def generate_overlay_path(row,year,filt=None):
            
    # determine if static variable
    if row['temporal resolution'] == 'static':
        return [row['path']],[row['path']]
    
    if row['temporal resolution'] == 'long term':
        perc_list = row['perc'].split(',')
        output_paths = [row['path'].replace('{perc}', perc) for perc in perc_list]
        return output_paths, output_paths
        
    # determine if the year is ahead of the availibility of the variable
    if year>int(row['end year']):
        year = int(row['end year'])
    if year<int(row['start year']):
        year = int(row['start year'])
    # determine if it's an annual variable or (bi)monthly variable
    if '{start_m}' not in row['path']:
        output_paths = [row['path'].replace('{year}',f'{int(year)}')]
    else:
        output_paths = []
        start_list = row['start_m'].split(', ')
        end_list = row['end_m'].split(', ')
        output_paths = [row['path'].replace('{year}',f'{int(year)}').replace('{start_m}',start_list[i]).replace('{end_m}',end_list[i]) for i in range(len(end_list))]
    
    if '{perc}' in row['path']:
        perc_list = row['perc'].split(',')
        output_paths = [p.replace('{perc}', perc) for p in output_paths for perc in perc_list]
        
    if (row['leap year'] == '1') & (year%4==0):
        output_paths = [p.replace('0228', '0229') if '0228' in p else p for p in output_paths]
    
    return output_paths, [i.replace(str(int(year)),'{year}') for i in output_paths]
    
def check_path(url):
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        # Check if the status code is not 200 (OK). You might want to specifically check for 404 or other error codes.
        if response.status_code == 404:
            print(f"{url} returned HTTP 404 Not Found")
            return url
        elif response.status_code != 200:
            print(f"{url} returned HTTP {response.status_code}")
            return url
        return None  # URL is fine (HTTP 200), or you might want to handle redirections (HTTP 3xx) separately if needed.
    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {str(e)}")
        return url
    
# # check function validity
# # generate paths
# paths = []
# for index,row in covar.iterrows():
#     paths.extend(generate_overlay_path(row,2000))
    
pathl = []
namel = []
year = 2000
for index,row in covar.iterrows():
    # if row['need update in overlay']=='1':
    #     paths, names = generate_overlay_path(row, year)
    #     pathl.extend(paths)
    #     namel.extend(names)
    paths, names = generate_overlay_path(row, year)
    pathl.extend(paths)
    namel.extend(names)
for i in pathl:
    check_path(i)

#### mend overlay on long term covars

In [11]:
path_stem = [i.split('/')[-1][0:-4] for i in pathl]
namel = [i.split('/')[-1][0:-4] for i in namel]
name_mapping = dict(zip(path_stem,namel))
    
df_overlay = df3035
        
ttprint(f'start overlaying for static, size: {len(df_overlay)}, column num: {len(pathl)}')
pathl = [Path(ii) for ii in pathl]
dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
temp = dfo.run()

[14:20:58] start overlaying for static, size: 394643, column num: 4


### overlay year by year
- divide soil data by year
- overlay the soil data in each year with corresponding covariates

In [7]:
drop = []
for i in df3035.columns:
    if 'wv' in i:
        drop.append(i)        
    if 'cropland' in i:
        drop.append(i)
df3035 = df3035.drop(columns = drop)
print(len(drop))

drop = []
for i in df3035.columns:
    if 'wv' in i:
        drop.append(i)        
    if 'cropland' in i:
        drop.append(i)
        
drop

42


[]

In [5]:
import warnings
warnings.filterwarnings("ignore")

# epsg 3035 overlay
co3035 = covar.loc[covar['epsg']=='3035']

# for year in np.arange(2000,2023,1):
for year in np.arange(2016, 2027, 1):
    pathl = []
    namel = []
    for index,row in co3035.iterrows():
        paths, names = generate_overlay_path(row, year)
        pathl.extend(paths)
        namel.extend(names)
        # if row['need update in overlay']=='1':
        #     paths, names = generate_overlay_path(row, year)
        #     pathl.extend(paths)
        #     namel.extend(names)
            
    # for iii in pathl:
    #     check_path(iii)

    path_stem = [i.split('/')[-1][0:-4] for i in pathl]
    namel = [i.split('/')[-1][0:-4] for i in namel]
    name_mapping = dict(zip(path_stem,namel))
    
    df_overlay = df3035.loc[df3035['time']==year]
    if len(df_overlay)==0:
        print(f'no data for year {year}')
        continue
        
    ttprint(f'start overlaying for year {str(int(year))}, size: {len(df_overlay)}, column num: {len(pathl)}')    
    pathl = [Path(ii) for ii in pathl]
    dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
    temp = dfo.run()
    
    temp = temp.rename(columns=name_mapping)
    temp=temp.drop(columns=['overlay_id'])
    # temp = pd.read_csv(f'/mnt/inca/soc_eu_model/overlay_intermediate/dft_{str(int(tt))}_3035.csv',index=False)
    temp.to_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_3035.csv',index=False)
    ttprint(f'finish overlaying for year {str(int(year))}')

[15:14:23] start overlaying for year 2016, size: 16177, column num: 441
[15:30:46] finish overlaying for year 2016
[15:30:46] start overlaying for year 2017, size: 15843, column num: 441
[15:45:30] finish overlaying for year 2017
[15:45:30] start overlaying for year 2018, size: 42918, column num: 441
[17:02:27] finish overlaying for year 2018
[17:02:27] start overlaying for year 2019, size: 15088, column num: 441
[17:17:50] finish overlaying for year 2019
[17:17:50] start overlaying for year 2020, size: 10200, column num: 441
[17:33:50] finish overlaying for year 2020
[17:33:50] start overlaying for year 2021, size: 10886, column num: 441
[17:42:59] finish overlaying for year 2021
[17:42:59] start overlaying for year 2022, size: 3294, column num: 441
[17:49:06] finish overlaying for year 2022
no data for year 2023
no data for year 2024
no data for year 2025
no data for year 2026


### assemble the overlayed annual datasets
- read in the overlayed soil data (with covariates) for each year
- combine them into a whole dataset

In [2]:
## read in 3035 datasets
mended = ''
tl = []
for year in np.arange(2000,2023,1):
    temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}{mended}_3035.csv',low_memory=False)
    print(f'{year}, {len(temp.columns)}, {len(temp)}')
    tl.append(temp)

df3035 = pd.concat(tl)
print(f'whole 3035, cols:{len(df3035.columns)}, size:{len(df3035)}')


2000, 476, 26304
2001, 476, 12777
2002, 476, 11718
2003, 476, 13497
2004, 476, 7649
2005, 476, 12987
2006, 476, 13075
2007, 476, 18561
2008, 476, 11983
2009, 476, 25942
2010, 476, 12330
2011, 476, 15374
2012, 476, 26062
2013, 476, 21428
2014, 476, 16922
2015, 476, 33628
2016, 476, 16177
2017, 476, 15843
2018, 476, 42918
2019, 476, 15088
2020, 476, 10200
2021, 476, 10886
2022, 476, 3294
whole 3035, cols:476, size:394643


In [3]:
col = 'cropland.extent_glad.interpolate_p_30m_s_{year}0101_{year}1231_eu_epsg.3035_v20240604'
for year in np.arange(2000,2023,1):
    temp = df3035.loc[df3035['time']==year]
    aaa = temp[col].isna().sum()/len(temp)
    print(aaa, year, temp[col].min())
    
df3035.loc[df3035[col].isna(),col] = 0

0.7101201338199513 2000 100.0
0.8049620411677233 2001 100.0
0.6483188257381806 2002 100.0
0.4095724975920575 2003 100.0
0.7522551967577461 2004 25.0
0.7871717871717872 2005 50.0
0.6450478011472275 2006 25.0
0.6639189698830882 2007 100.0
0.6091129099557707 2008 25.0
0.5784442217253875 2009 50.0
0.5128953771289537 2010 25.0
0.5467022245349291 2011 100.0
0.6755429360755122 2012 25.0
0.4654657457532201 2013 50.0
0.6197848954024348 2014 25.0
0.578684429641965 2015 100.0
0.6435062125239538 2016 25.0
0.4782553809253298 2017 50.0
0.5796868446805536 2018 25.0
0.7391304347826086 2019 100.0
0.7472549019607844 2020 100.0
0.6805989344111704 2021 100.0
0.8269581056466302 2022 100.0


In [5]:
dff = df3035
meta_list = ['id', 'lat', 'lon', 'time', 'hzn_top', 'hzn_btm', 'ref', 'nuts0', 'oc',
       'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay', 'silt', 'sand', 'caco3',
       'N', 'K', 'P', 'CEC', 'EC', 'oc_qa', 'N_qa', 'caco3_qa',
       'bulk_density_qa', 'clay_qa', 'silt_qa', 'sand_qa', 'ph_h2o_qa',
       'ph_cacl2_qa', 'P_qa', 'K_qa', 'EC_qa', 'CEC_qa', 'geometry']
### check covariates availability
drop_list = []
for col in dff.columns:
    if col in meta_list:
        continue
    if (dff[col].isna().sum()/len(dff))>0.02:
        
        if 'longterm' not in col:
            print(col, dff[col].isna().sum())
            drop_list.append(col)
        
print(f'remove covariates with more than 2% data unavailable')
dff = dff.drop(columns = drop_list)

fgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 264869
fcf_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 198538
lgd_chelsa_m_1km_s_19810101_20101231_eu_epsg.3035_v20240531 264869
surface.water_jrc.gswe_p_30m_s_2000_2018_go_epsg.4326_v01232022 394643
remove covariates with more than 2% data unavailable


In [6]:
dff = dff.drop(columns=['geometry'])
# dff.to_csv(f'{folder}/data/001_covar_overlayed.csv',index=False)

### Assign spatial blocking ID

In [7]:
# create a tiling system first
from eumap.parallel import TilingProcessing
from pathlib import Path
import rasterio
from shapely.geometry import box
import numpy as np
import pandas as pd
import geopandas as gpd

# # create tiling system
# raster_layer_fn = f'http://192.168.1.30:8333/ai4sh-landmasked/ndvi/ndvi_glad.landast.ard2.seasconv.m.yearly_p75_30m_s_20220101_20221231_eu_epsg.3035_v20231127.tif'
# ds = rasterio.open(raster_layer_fn)
# tiles_size = ds.transform[0] * 1000 # 30m -> 30km
# tiling_system = TilingProcessing.generate_tiles(tiles_size, extent=ds.bounds, crs=ds.crs, raster_layer_fn=raster_layer_fn)
# tiling_system = tiling_system.to_crs("EPSG:4326")
# tiling_system = tiling_system[['tile_id','geometry']]
# tiling_system.to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg',  driver="GPKG")
# # tiling_system[tiling_system['raster_mode_count'] > 0].to_file('/mnt/inca/soc_eu_model/data/000_tile_eu4326.gpkg.gpkg',  driver="GPKG")


from shapely.geometry import Point
tiles = gpd.read_file(f'{folder}/data/000_tiles.rolf_epsg.3035.gpkg')

# dff = pd.read_csv(f'{folder}/data/001_covar_overlayed.csv',low_memory=False)
gdf = gpd.GeoDataFrame(dff, geometry=gpd.points_from_xy(dff.lon, dff.lat))
gdf.crs = 'EPSG:4326'
gdf = gdf.to_crs(tiles.crs)

# join spatial
joined_gdf = gpd.sjoin(gdf, tiles, how="left", op='within')
joined_gdf = joined_gdf.drop(columns=['geometry', 'index_right', 'left', 'top', 'right', 'bottom','row_index', 'col_index'])
joined_gdf = joined_gdf.rename(columns={'id_right':'tile_id','id_left':'id'})

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [9]:
joined_gdf.to_csv(f'{folder}/data/001_covar_overlayed.csv',index=False)

In [8]:
joined_gdf

,id,lat,lon,time,hzn_top,hzn_btm,ref,nuts0,oc,ph_h2o,...,max.curv.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240501,dtm.bareearth_ensemble_p10_30m_s_20000101_20221231_eu_epsg.3035_v20240424,flow.accum.bareearth_ensemble_m_60m_s_20000101_20221231_eu_epsg.3035_v20240501,neg.openess.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240424,geomorphon.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240424,pos.openess.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240424,easterness.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240424,northness.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240424,min.curv.bareearth_ensemble_m_30m_s_20000101_20221231_eu_epsg.3035_v20240424,tile_id
0,1-1,40.871480,-7.049093,2000.0,0.0,30.0,portugal.infosolo,PT,6.70,5.2,...,514.0,6161.0,11695.0,151.0,6.0,151.0,-90.0,-43.0,-306.0,636.0
1,1-2,40.871480,-7.049093,2000.0,30.0,55.0,portugal.infosolo,PT,3.90,5.5,...,514.0,6161.0,11695.0,151.0,6.0,151.0,-90.0,-43.0,-306.0,636.0
2,2-3,40.583860,-6.952317,2000.0,0.0,13.0,portugal.infosolo,PT,5.90,4.9,...,-28.0,6416.0,24814.0,153.0,6.0,147.0,-4.0,100.0,-585.0,636.0
3,2-4,40.583860,-6.952317,2000.0,13.0,38.0,portugal.infosolo,PT,6.20,5.2,...,-28.0,6416.0,24814.0,153.0,6.0,147.0,-4.0,100.0,-585.0,636.0
4,3-5,40.756090,-6.862586,2000.0,0.0,25.0,portugal.infosolo,PT,5.40,5.4,...,17.0,7007.0,42397.0,155.0,6.0,154.0,-53.0,85.0,-341.0,636.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289,Diputación de Palencia - DP220201,41.935616,-4.513888,2022.0,0.0,25.0,Castilla.y.Leon,ES,6.55,NaN,...,35.0,7192.0,26815.0,157.0,1.0,154.0,99.0,-11.0,-168.0,711.0
3290,Diputación de Palencia - DP220202,41.935616,-4.513888,2022.0,0.0,25.0,Castilla.y.Leon,ES,7.35,NaN,...,35.0,7192.0,26815.0,157.0,1.0,154.0,99.0,-11.0,-168.0,711.0
3291,Diputación de Palencia - DP220030,41.775509,-3.959406,2022.0,0.0,25.0,Castilla.y.Leon,ES,4.30,NaN,...,50.0,8504.0,13133.0,156.0,1.0,155.0,99.0,16.0,-400.0,712.0
3292,Diputación de Palencia - DP220090,42.323577,-4.312706,2022.0,0.0,25.0,Castilla.y.Leon,ES,3.60,NaN,...,58.0,7935.0,29635.0,156.0,6.0,155.0,86.0,50.0,-96.0,711.0


### epsg 4326

In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

# # epsg 4326 overlay
# co4326 = covar.loc[covar['epsg']=='4326']
# path_ori = [i.split('/')[-1][0:-4] for i in co4326['path']]

# for year in np.arange(2000,2024,1):
#     pathl = []
#     namel = []
#     for index,row in co4326.iterrows():
#         paths, names = generate_overlay_path(row, year)
#         pathl.extend(paths)
#         namel.extend(names)
#         # path4326.extend(generate_overlay_path(row,year))
        
# #     for i in pathl:
# #         check_path(i)
        
#     path_stem = [i.split('/')[-1][0:-4] for i in pathl]
#     namel = [i.split('/')[-1][0:-4] for i in namel]
#     name_mapping = dict(zip(path_stem,namel))
    
#     df_overlay = df4326.loc[df4326['time']==year]
#     if len(df_overlay)==0:
#         print(f'no data for year {year}')
#         continue
        
#     ttprint(f'start overlaying for year {str(int(year))}, size: {len(df_overlay)}')
#     pathl = [Path(ii) for ii in pathl]
#     dfo = SpaceOverlay(df_overlay, fn_layers=pathl, max_workers=90, verbose=False)
#     temp = dfo.run()
    
#     temp = temp.rename(columns=name_mapping)
#     temp=temp.drop(columns=['overlay_id'])
#     # temp = pd.read_csv(f'/mnt/inca/soc_eu_model/overlay_intermediate/dft_{str(int(tt))}_4326.csv',index=False)
#     temp.to_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',index=False)
#     ttprint(f'finish overlaying for year {str(int(year))}')


# # read in 4326 datasets
# tl = []

# for year in np.arange(2000,2023,1):
#     temp = pd.read_csv(f'{folder}/overlay_intermediate/dft_{str(int(year))}_4326.csv',low_memory=False)
#     temp = temp.rename(columns=name_mapping)
#     print(f'{year}, {len(temp.columns)}, {len(temp)}')
#     tl.append(temp)

# df4326 = pd.concat(tl)
# print(f'whole 4326, {len(df4326.columns)}, {len(df4326)}')


# # merge to merge
# cols_list = df3035.columns.values.tolist()
# meta_list = ['lat', 'lon', 'oc', 'ph_h2o', 'ph_cacl2', 'bulk_density', 'clay','silt', 'sand', 'caco3','N',
#              'K', 'P', 'CEC', 'EC', 'nuts0', 'time','hzn_top','hzn_btm','ref','sample_id']
# dff = pd.merge(df3035,df4326,on = meta_list, how='inner')
